# _utils.base

In [ ]:
#|default_exp _utils.base

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import repoyard._utils.base as this_module

In [ ]:
#|export
import subprocess
import shlex
import json
import asyncio
from enum import Enum
from repoyard import const
from pathlib import Path
from typing import Any, Coroutine

import repoyard.config

In [ ]:
#|hide
show_doc(this_module.get_repo_index_name_from_sub_path)

## get_repo_index_name_from_sub_path

```python
get_repo_index_name_from_sub_path(
    config: Config,
    sub_path: str
) -> pathlib.Path | None
```

Get the index name of a synced repo from a path inside of the repo.

---


In [ ]:
#|export
def get_repo_index_name_from_sub_path(
    config: repoyard.config.Config,
    sub_path: str,
) -> Path|None:
    """
    Get the index name of a synced repo from a path inside of the repo.
    """
    sub_path = Path(sub_path).expanduser().resolve() # Need to resolve to replace symlinks
    is_in_local_store_path = sub_path.is_relative_to(config.user_repos_path)
    
    if not is_in_local_store_path:
        return None
    
    rel_path = sub_path.relative_to(config.user_repos_path)
    
    if config.user_repos_path.as_posix() == sub_path.as_posix(): # The path is not inside a repo but is in the repo store root
        return None
    
    repo_index_name = rel_path.parts[0]
    return repo_index_name

In [ ]:
#|hide
show_doc(this_module.get_hostname)

## get_hostname

```python
get_hostname()
```

---


In [ ]:
#|export
import platform
import subprocess

def get_hostname():
    system = platform.system()
    hostname = None
    if system == "Darwin":
        # Mac
        try:
            result = subprocess.run(["scutil", "--get", "ComputerName"], capture_output=True, text=True, check=True)
            hostname = result.stdout.strip()
        except Exception:
            hostname = None
    if hostname is None:
        hostname = platform.node()
    return hostname

In [ ]:
#|hide
show_doc(this_module.run_fzf)

## run_fzf

```python
run_fzf(terms: list[str], disp_terms: list[str] | None)
```

Launches the fzf command-line fuzzy finder with a list of terms and returns
the selected term.

Parameters:
terms (List[str]): A list of strings to be presented to fzf for selection.

Returns:
str or None: The selected string from fzf, or None if no selection was made
or if fzf encountered an error.

Raises:
RuntimeError: If fzf is not installed or not found in the system PATH.

---


In [ ]:
#|export
def run_fzf(terms: list[str], disp_terms: list[str]|None=None):
    """
    Launches the fzf command-line fuzzy finder with a list of terms and returns
    the selected term.

    Parameters:
    terms (List[str]): A list of strings to be presented to fzf for selection.

    Returns:
    str or None: The selected string from fzf, or None if no selection was made
    or if fzf encountered an error.

    Raises:
    RuntimeError: If fzf is not installed or not found in the system PATH.
    """
    import subprocess
    if disp_terms is None: disp_terms = terms
    try:
        # Launch fzf with the list of strings
        result = subprocess.run(
            ['fzf'],
            input='\n'.join(disp_terms),
            text=True,
            capture_output=True
        )
        res_term = result.stdout.strip()
        term_index = [t.strip() for t in disp_terms].index(res_term)
        sel_term = terms[term_index]
        # Return the selected string or None if no selection was made
        if result.returncode != 0: 
            return None, None
        else: 
            return term_index, sel_term
    except FileNotFoundError:
        raise RuntimeError("fzf is not installed or not found in PATH.")

In [ ]:
#|hide
show_doc(this_module.check_last_time_modified)

## check_last_time_modified

```python
check_last_time_modified(path: str | pathlib.Path) -> float | None
```

---


In [ ]:
#|export
def check_last_time_modified(path: str | Path) -> float | None:
    import os
    from datetime import datetime, timezone
    path = Path(path).expanduser().resolve()
    
    if path.is_file():
        max_mtime = path.stat().st_mtime
    else:
        max_mtime = None
        stack = [str(path)]
        
        while stack:
            current = stack.pop()
            try:
                with os.scandir(current) as entries:
                    for entry in entries:
                        if entry.is_file(follow_symlinks=False):
                            try:
                                stat_result = entry.stat()
                                mtime = stat_result.st_mtime
                                if max_mtime is None or mtime > max_mtime:
                                    max_mtime = mtime
                            except (OSError, PermissionError):
                                continue
                        elif entry.is_dir(follow_symlinks=False):
                            stack.append(entry.path)
            except (OSError, PermissionError):
                continue
    
    return datetime.fromtimestamp(max_mtime, tz=timezone.utc) if max_mtime is not None else None

In [ ]:
#|hide
show_doc(this_module.run_cmd_async)

## run_cmd_async *(async)*

```python
run_cmd_async(cmd: list[str]) -> Popen
```

---


In [ ]:
#|export
async def run_cmd_async(cmd: list[str]) -> subprocess.Popen:
    proc = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    stdout, stderr = await proc.communicate()
    stdout = stdout.decode('utf-8')
    stderr = stderr.decode('utf-8')
    return proc.returncode, stdout, stderr

In [ ]:
await run_cmd_async(['echo', 'hello', 'world'])

(0, 'hello world\n', '')

In [ ]:
#|hide
show_doc(this_module.async_throttler)

## async_throttler *(async)*

```python
async_throttler(
    coros: list[typing.Coroutine],
    max_concurrency: int,
    timeout: float | None
) -> list[typing.Any]
```

Throttle a list of coroutines to run concurrently.

---


In [ ]:
#|export
async def async_throttler(
    coros: list[Coroutine],
    max_concurrency: int,
    timeout: float | None = None,
) -> list[Any]:
    """
    Throttle a list of coroutines to run concurrently.
    """

    sem = asyncio.Semaphore(max_concurrency)
    
    async def _task(coro: Coroutine) -> Any:
        async with sem:
            try:
                if timeout is None:
                    return await coro
                else:
                    return await asyncio.wait_for(coro, timeout)
            except asyncio.TimeoutError as e:
                return e
            except Exception as e:
                return e

    tasks = [_task(coro) for coro in coros]
    res = await asyncio.gather(*tasks, return_exceptions=True)
    for r in res:
        if isinstance(r, Exception):
            raise r
    return res

In [ ]:
async def test_task():
    await asyncio.sleep(0.1)

coros = [test_task() for _ in range(10)]
res = await async_throttler(coros, max_concurrency=2)

In [ ]:
#|hide
show_doc(this_module.is_in_event_loop)

## is_in_event_loop

```python
is_in_event_loop()
```

---


In [ ]:
#|export
def is_in_event_loop():
    try:
        asyncio.get_running_loop()
        return True
    except RuntimeError:
        return False

In [ ]:
#|hide
show_doc(this_module.enable_soft_interruption)

## enable_soft_interruption

```python
enable_soft_interruption()
```

---


In [ ]:
#|export
import signal
import sys

_interrupted = False
_interrupt_count = 0

class SoftInterruption(Exception):
    pass

def _soft_interruption_handler(signum, frame):
    global _interrupted, _interrupt_count
    _interrupt_count += 1
    sig_name = signal.Signals(signum).name

    if _interrupt_count < const.SOFT_INTERRUPT_COUNT:
        print(f"\nWARNING: {sig_name} received ({_interrupt_count}/3) — "
              f"will stop after the current operation.")
        _interrupted = True
    else:
        print(f"\n{sig_name} received 3 times — exiting immediately.")
        sys.exit(1)   # or: raise KeyboardInterrupt

def enable_soft_interruption():
    signal.signal(signal.SIGINT, _soft_interruption_handler)  # Ctrl-C
    signal.signal(signal.SIGTERM, _soft_interruption_handler)  # shutdown
    signal.signal(signal.SIGHUP, _soft_interruption_handler)  # logout / terminal closed

def check_interrupted():
    global _interrupted
    return _interrupted

In [ ]:
p = Path("/Users/lukastk/dev/20251109_000000_7GfJI__repoyard")

import os

files = []
for path, dirs, filenames in os.walk(p):
    for name in filenames:
        files.append(os.path.join(path, name))

print(len(files))

0


In [ ]:
#|hide
show_doc(this_module.count_files_in_dir)

## count_files_in_dir

```python
count_files_in_dir(path: Path) -> int
```

---


In [ ]:
#|export
def count_files_in_dir(path: Path) -> int:
    num_files = 0
    for path, dirs, filenames in os.walk(path):
        num_files += len(filenames)
    return num_files